In [1]:
import numpy as np
import tensorflow as tf
from tensorflow import keras

In [2]:
from sklearn.datasets import fetch_california_housing
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

In [3]:
housing = fetch_california_housing()
X, y = housing.data, housing.target

In [4]:
X_train_full, X_test, y_train_full, y_test = train_test_split(X, y, random_state=42)
X_train, X_val, y_train, y_val = train_test_split(X_train_full, y_train_full, random_state=42)

In [5]:
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)
X_val = scaler.transform(X_val)

In [6]:
np.random.seed(42)
tf.random.set_seed(42)

# Functional API

In [7]:
input_ = keras.layers.Input(shape=X_train.shape[1:])
hidden_1 = keras.layers.Dense(30, activation="relu")(input_)
hidden_2 = keras.layers.Dense(30, activation="relu")(hidden_1)
concat = keras.layers.Concatenate()([input_, hidden_2])
output = keras.layers.Dense(1)(concat)
model = keras.models.Model(inputs=[input_], outputs=[output])

In [8]:
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 8)]          0           []                               
                                                                                                  
 dense (Dense)                  (None, 30)           270         ['input_1[0][0]']                
                                                                                                  
 dense_1 (Dense)                (None, 30)           930         ['dense[0][0]']                  
                                                                                                  
 concatenate (Concatenate)      (None, 38)           0           ['input_1[0][0]',                
                                                                  'dense_1[0][0]']            

In [9]:
model.compile(loss="mean_squared_error",optimizer=keras.optimizers.SGD(learning_rate=1e-3))
history = model.fit(X_train, y_train, epochs=20, validation_data=(X_val, y_val))
mse_test = model.evaluate(X_test, y_test)

Epoch 1/20
363/363 [==============================] - 1s 1ms/step - loss: 1.7767 - val_loss: 12.9986
Epoch 2/20
363/363 [==============================] - 0s 957us/step - loss: 0.7627 - val_loss: 0.6218
Epoch 3/20
363/363 [==============================] - 0s 936us/step - loss: 0.6061 - val_loss: 0.6448
Epoch 4/20
363/363 [==============================] - 0s 935us/step - loss: 0.5770 - val_loss: 0.5414
Epoch 5/20
363/363 [==============================] - 0s 935us/step - loss: 0.5514 - val_loss: 0.5247
Epoch 6/20
363/363 [==============================] - 0s 941us/step - loss: 0.5299 - val_loss: 0.5118
Epoch 7/20
363/363 [==============================] - 0s 934us/step - loss: 0.5128 - val_loss: 0.5089
Epoch 8/20
363/363 [==============================] - 0s 943us/step - loss: 0.4991 - val_loss: 0.4954
Epoch 9/20
363/363 [==============================] - 0s 957us/step - loss: 0.4848 - val_loss: 0.4635
Epoch 10/20
363/363 [==============================] - 0s 936us/step - loss: 0.4735

In [10]:
mse_test

0.40746474266052246

In [11]:
input_a = keras.layers.Input(shape=[5], name="wide_input")
input_b = keras.layers.Input(shape=[6], name="deep_input")
hidden_1 = keras.layers.Dense(30, activation="relu")(input_b)
hidden_2 = keras.layers.Dense(30, activation="relu")(hidden_1)
conc = keras.layers.Concatenate()([input_a, hidden_2])
output = keras.layers.Dense(1, name="output")(conc)
model = keras.Model(inputs=[input_a, input_b], outputs=output)

In [12]:
model.compile(loss="mse", optimizer=keras.optimizers.SGD(learning_rate=1e-3))

X_train_A, X_train_B = X_train[:,:5], X_train[:,2:]
X_test_A, X_test_B = X_test[:,:5], X_test[:,2:]
X_val_A, X_val_B = X_val[:,:5], X_val[:,2:]
X_new_A, X_new_B = X_test_A[:3], X_test_B[:3]

history = model.fit((X_train_A, X_train_B), y_train, epochs=20,
                    validation_data=((X_val_A, X_val_B), y_val))
mse_test = model.evaluate((X_test_A,X_test_B), y_test)

Epoch 1/20
363/363 [==============================] - 1s 1ms/step - loss: 2.0125 - val_loss: 0.8598
Epoch 2/20
363/363 [==============================] - 0s 963us/step - loss: 0.7278 - val_loss: 0.6542
Epoch 3/20
363/363 [==============================] - 0s 979us/step - loss: 0.6458 - val_loss: 0.6060
Epoch 4/20
363/363 [==============================] - 0s 1ms/step - loss: 0.6091 - val_loss: 0.5615
Epoch 5/20
363/363 [==============================] - 0s 985us/step - loss: 0.5835 - val_loss: 0.5374
Epoch 6/20
363/363 [==============================] - 0s 985us/step - loss: 0.5614 - val_loss: 0.5150
Epoch 7/20
363/363 [==============================] - 0s 1ms/step - loss: 0.5442 - val_loss: 0.5006
Epoch 8/20
363/363 [==============================] - 0s 985us/step - loss: 0.5291 - val_loss: 0.4861
Epoch 9/20
363/363 [==============================] - 0s 982us/step - loss: 0.5157 - val_loss: 0.4725
Epoch 10/20
363/363 [==============================] - 0s 964us/step - loss: 0.5044 - va

In [13]:
input_A = keras.layers.Input(shape=[5], name="wide_input") 
input_B = keras.layers.Input(shape=[6], name="deep_input")
hidden_1 = keras.layers.Dense(30, activation="relu")(input_B)
hidden_2 = keras.layers.Dense(30, activation="relu")(hidden_1)
concat = keras.layers.Concatenate()([hidden_1, input_A])
output = keras.layers.Dense(1, name="main_output")(concat)
aux = keras.layers.Dense(1, name="Aux_output")(hidden_2)

model = keras.Model(inputs=[input_A, input_B], outputs=[output, aux])
model.compile(loss=["mse","mse"], loss_weights=[0.9, 0.1], optimizer=keras.optimizers.SGD(learning_rate=1e-3))

In [14]:
history = model.fit([X_train_A, X_train_B], [y_train, y_train], epochs=20, validation_data=([X_val_A, X_val_B], [y_val,y_val]))

Epoch 1/20
363/363 [==============================] - 1s 1ms/step - loss: 2.9620 - main_output_loss: 2.6077 - Aux_output_loss: 6.1509 - val_loss: 1.9989 - val_main_output_loss: 1.6752 - val_Aux_output_loss: 4.9127
Epoch 2/20
363/363 [==============================] - 0s 1ms/step - loss: 1.2129 - main_output_loss: 0.9095 - Aux_output_loss: 3.9433 - val_loss: 1.2201 - val_main_output_loss: 0.8097 - val_Aux_output_loss: 4.9141
Epoch 3/20
363/363 [==============================] - 0s 1ms/step - loss: 0.9029 - main_output_loss: 0.7021 - Aux_output_loss: 2.7104 - val_loss: 1.1281 - val_main_output_loss: 0.6592 - val_Aux_output_loss: 5.3481
Epoch 4/20
363/363 [==============================] - 0s 1ms/step - loss: 0.7802 - main_output_loss: 0.6384 - Aux_output_loss: 2.0567 - val_loss: 1.0741 - val_main_output_loss: 0.6102 - val_Aux_output_loss: 5.2492
Epoch 5/20
363/363 [==============================] - 0s 1ms/step - loss: 0.7117 - main_output_loss: 0.6019 - Aux_output_loss: 1.7006 - val_loss

# Subclassing API

In [15]:
class WideAndDeepModel(keras.models.Model):
    def __init__(self, units, activation, **kwargs):
        super().__init__(**kwargs)
        self.hidden1 = keras.layers.Dense(units, activation=activation)
        self.hidden2 = keras.layers.Dense(units, activation=activation)
        self.main_output = keras.layers.Dense(1)
        self.aux_output = keras.layers.Dense(1)
    
    def call(self, inputs):
        input_A, input_B = inputs
        hidden1 = self.hidden1(input_B)
        hidden2 = self.hidden2(hidden1)
        concat = keras.layers.Concatenate()([input_A, hidden2])
        main_output = self.main_output(concat)
        aux_output = self.aux_output(hidden2)
        return main_output, aux_output

In [16]:
model = WideAndDeepModel(units=30, activation="relu")

In [17]:
model.compile(loss="mse", loss_weights=[0.9, 0.1], optimizer=keras.optimizers.SGD(learning_rate=1e-3))
history = model.fit((X_train_A, X_train_B), (y_train, y_train), epochs=10,
                    validation_data=((X_val_A, X_val_B), (y_val, y_val)))

Epoch 1/10
363/363 [==============================] - 1s 1ms/step - loss: 2.1362 - output_1_loss: 1.8201 - output_2_loss: 4.9810 - val_loss: 3.2150 - val_output_1_loss: 2.9510 - val_output_2_loss: 5.5911
Epoch 2/10
363/363 [==============================] - 0s 1ms/step - loss: 0.9772 - output_1_loss: 0.7419 - output_2_loss: 3.0944 - val_loss: 1.5983 - val_output_1_loss: 1.0903 - val_output_2_loss: 6.1704
Epoch 3/10
363/363 [==============================] - 0s 1ms/step - loss: 0.7844 - output_1_loss: 0.6266 - output_2_loss: 2.2041 - val_loss: 1.1590 - val_output_1_loss: 0.6030 - val_output_2_loss: 6.1629
Epoch 4/10
363/363 [==============================] - 0s 1ms/step - loss: 0.7000 - output_1_loss: 0.5712 - output_2_loss: 1.8591 - val_loss: 1.0323 - val_output_1_loss: 0.5276 - val_output_2_loss: 5.5747
Epoch 5/10
363/363 [==============================] - 0s 1ms/step - loss: 0.6501 - output_1_loss: 0.5343 - output_2_loss: 1.6916 - val_loss: 0.9293 - val_output_1_loss: 0.4939 - val_ou

In [18]:
eval_results = model.evaluate([X_test_A, X_test_B],[y_test, y_test])

162/162 [==============================] - 0s 724us/step - loss: 0.5451 - output_1_loss: 0.4518 - output_2_loss: 1.3849


# Saving and Restoring

In [19]:
model = keras.models.Sequential([
    keras.layers.Dense(30, activation="relu", input_shape=[8]),
    keras.layers.Dense(30, activation="relu"),
    keras.layers.Dense(1)
])

In [20]:
model.compile(loss="mse", optimizer=keras.optimizers.SGD(learning_rate=1e-3))
model.fit(X_train, y_train, epochs=30, validation_data=(X_val, y_val))
mse_test = model.evaluate(X_test, y_test)
model.weights

Epoch 1/30
363/363 [==============================] - 1s 1ms/step - loss: 1.4622 - val_loss: 0.7607
Epoch 2/30
363/363 [==============================] - 0s 1ms/step - loss: 0.6810 - val_loss: 0.6462
Epoch 3/30
363/363 [==============================] - 0s 970us/step - loss: 0.6054 - val_loss: 0.5799
Epoch 4/30
363/363 [==============================] - 0s 949us/step - loss: 0.5572 - val_loss: 0.5208
Epoch 5/30
363/363 [==============================] - 0s 1ms/step - loss: 0.5232 - val_loss: 0.4855
Epoch 6/30
363/363 [==============================] - 0s 1ms/step - loss: 0.4956 - val_loss: 0.4577
Epoch 7/30
363/363 [==============================] - 0s 1ms/step - loss: 0.4748 - val_loss: 0.4381
Epoch 8/30
363/363 [==============================] - 0s 1ms/step - loss: 0.4580 - val_loss: 0.4246
Epoch 9/30
363/363 [==============================] - 0s 1ms/step - loss: 0.4452 - val_loss: 0.4142
Epoch 10/30
363/363 [==============================] - 0s 1ms/step - loss: 0.4350 - val_loss: 0.

[<tf.Variable 'dense_11/kernel:0' shape=(8, 30) dtype=float32, numpy=
 array([[-0.12087043,  0.28481257,  0.31336507, -0.37000388, -0.36882052,
         -0.11483479, -0.27037796, -0.14777307,  0.30924073,  0.4241558 ,
         -0.37212482,  0.13658507,  0.22353883, -0.31132403, -0.38484865,
         -0.27470568, -0.04676881, -0.13332978,  0.1833537 , -0.18632115,
          0.3080944 , -0.05635306, -0.06391677,  0.38140365, -0.15410374,
         -0.4486882 ,  0.28715903,  0.15308477,  0.19132423, -0.05248149],
        [ 0.19933884,  0.2886085 , -0.01164274,  0.00294578, -0.12389822,
         -0.34602547, -0.06061186,  0.08724023, -0.09141105,  0.27745998,
         -0.30206764,  0.09970113,  0.23616503, -0.19468474, -0.17102586,
         -0.1511357 , -0.2374795 , -0.25580958,  0.11415473,  0.12594832,
         -0.046075  ,  0.04251806,  0.23041263,  0.1990844 ,  0.04652935,
         -0.17252716,  0.21871778,  0.26639676,  0.01577725, -0.3525061 ],
        [-0.43448058, -0.15865672, -0.10

In [21]:
model.save("my_keras_model.h5")

In [22]:
model.save_weights("my_keras_weights.ckpt")

In [23]:
model.load_weights("my_keras_weights.ckpt")

# Callbacks during Training

In [24]:
keras.backend.clear_session()
np.random.seed(42)
tf.random.set_seed(42)

In [25]:
model = keras.models.Sequential([
    keras.layers.Dense(30, activation="relu", input_shape=[8]),
    keras.layers.Dense(30, activation="relu"),
    keras.layers.Dense(1)
])

In [26]:
from keras.callbacks import ModelCheckpoint, EarlyStopping

In [27]:
model.compile(loss="mse", optimizer=keras.optimizers.SGD(learning_rate=1e-3))
checkpoint_cb  = ModelCheckpoint("my_keras_model.h5", save_best_only=True)
model.fit(X_train, y_train, epochs=20, validation_data=(X_val, y_val), callbacks=[checkpoint_cb])

Epoch 1/20
363/363 [==============================] - 1s 1ms/step - loss: 2.2130 - val_loss: 1.1679
Epoch 2/20
363/363 [==============================] - 0s 1ms/step - loss: 0.7856 - val_loss: 0.6840
Epoch 3/20
363/363 [==============================] - 0s 1ms/step - loss: 0.6924 - val_loss: 0.6272
Epoch 4/20
363/363 [==============================] - 0s 964us/step - loss: 0.6432 - val_loss: 0.5987
Epoch 5/20
363/363 [==============================] - 0s 956us/step - loss: 0.6068 - val_loss: 0.5549
Epoch 6/20
363/363 [==============================] - 0s 961us/step - loss: 0.5737 - val_loss: 0.5300
Epoch 7/20
363/363 [==============================] - 0s 992us/step - loss: 0.5475 - val_loss: 0.5029
Epoch 8/20
363/363 [==============================] - 0s 979us/step - loss: 0.5242 - val_loss: 0.4806
Epoch 9/20
363/363 [==============================] - 0s 1ms/step - loss: 0.5043 - val_loss: 0.4638
Epoch 10/20
363/363 [==============================] - 0s 956us/step - loss: 0.4875 - val_

In [28]:
model.compile(loss="mse", optimizer=keras.optimizers.SGD(learning_rate=1e-3))
early_stopping_cb = keras.callbacks.EarlyStopping(patience=10, restore_best_weights=True)
history = model.fit(X_train, y_train, epochs=20, validation_data=(X_val, y_val), callbacks=[early_stopping_cb] )
history.history["loss"]

Epoch 1/20
363/363 [==============================] - 1s 1ms/step - loss: 0.4086 - val_loss: 0.3860
Epoch 2/20
363/363 [==============================] - 0s 988us/step - loss: 0.4059 - val_loss: 0.4052
Epoch 3/20
363/363 [==============================] - 0s 1ms/step - loss: 0.4033 - val_loss: 0.3928
Epoch 4/20
363/363 [==============================] - 0s 1ms/step - loss: 0.4004 - val_loss: 0.3975
Epoch 5/20
363/363 [==============================] - 0s 997us/step - loss: 0.3988 - val_loss: 0.3750
Epoch 6/20
363/363 [==============================] - 0s 1ms/step - loss: 0.3961 - val_loss: 0.4029
Epoch 7/20
363/363 [==============================] - 0s 1ms/step - loss: 0.3948 - val_loss: 0.3772
Epoch 8/20
363/363 [==============================] - 0s 1ms/step - loss: 0.3928 - val_loss: 0.3700
Epoch 9/20
363/363 [==============================] - 0s 1ms/step - loss: 0.3909 - val_loss: 0.3701
Epoch 10/20
363/363 [==============================] - 0s 1ms/step - loss: 0.3892 - val_loss: 0.

[0.40860384702682495,
 0.4058571457862854,
 0.40334171056747437,
 0.40038663148880005,
 0.39876630902290344,
 0.3960975408554077,
 0.39479565620422363,
 0.39277562499046326,
 0.3909110724925995,
 0.3891757130622864,
 0.3880283236503601,
 0.38614341616630554,
 0.384860098361969,
 0.3836929500102997,
 0.3821837604045868,
 0.38092494010925293,
 0.3795040547847748,
 0.3788672089576721,
 0.376993864774704,
 0.37634947896003723]

# TensorBoard

In [29]:
import os

In [30]:
root_logdir = os.path.join(os.curdir, "my_logs")

In [31]:
def get_run_logdir():
    import time
    run_id = time.strftime("run_%Y_%m_%d-%H_%M_%S")
    return os.path.join(root_logdir, run_id)

run_logdir = get_run_logdir()
run_logdir

'.\\my_logs\\run_2023_04_27-09_45_23'

In [32]:
keras.backend.clear_session()
np.random.seed(42)
tf.random.set_seed(42)

In [33]:
model = keras.models.Sequential([
    keras.layers.Dense(30, activation="relu", input_shape=[8]),
    keras.layers.Dense(30, activation="relu"),
    keras.layers.Dense(1)
])

In [34]:
model.compile(loss="mse", optimizer=keras.optimizers.SGD(learning_rate=1e-3))

In [35]:
tensorboard_cb = keras.callbacks.TensorBoard(run_logdir)
checkpoint_cb = keras.callbacks.ModelCheckpoint("my_keras.model.h5", save_best_only=True)

history = model.fit(X_train, y_train, epochs=30,
                    validation_data=(X_val, y_val),
                   callbacks=[tensorboard_cb, checkpoint_cb])

Epoch 1/30
363/363 [==============================] - 1s 2ms/step - loss: 2.0740 - val_loss: 1.9296
Epoch 2/30
363/363 [==============================] - 1s 2ms/step - loss: 0.8635 - val_loss: 0.7333
Epoch 3/30
363/363 [==============================] - 1s 1ms/step - loss: 0.7315 - val_loss: 0.7230
Epoch 4/30
363/363 [==============================] - 0s 1ms/step - loss: 0.6672 - val_loss: 0.6655
Epoch 5/30
363/363 [==============================] - 1s 1ms/step - loss: 0.6204 - val_loss: 0.6131
Epoch 6/30
363/363 [==============================] - 0s 1ms/step - loss: 0.5818 - val_loss: 0.5919
Epoch 7/30
363/363 [==============================] - 0s 1ms/step - loss: 0.5508 - val_loss: 0.5675
Epoch 8/30
363/363 [==============================] - 0s 1ms/step - loss: 0.5245 - val_loss: 0.5039
Epoch 9/30
363/363 [==============================] - 0s 1ms/step - loss: 0.5014 - val_loss: 0.4797
Epoch 10/30
363/363 [==============================] - 0s 1ms/step - loss: 0.4820 - val_loss: 0.4593

# Fine-Tuning NN Hyperparameters

In [36]:
keras.backend.clear_session()
np.random.seed(42)
tf.random.set_seed(42)

In [37]:
def build_model(n_hidden=1, n_neurons=30, learning_rate=3e-3, input_shape=[8]):
    model = keras.models.Sequential()
    model.add(keras.layers.InputLayer(input_shape=input_shape))
    for i in range(n_hidden):
        model.add(keras.layers.Dense(n_neurons, activation="relu"))
    optimizer = keras.optimizers.SGD(learning_rate=learning_rate)
    model.compile(loss="mse", optimizer=optimizer)
    return model

In [38]:
from scikeras.wrappers import KerasRegressor

In [39]:
ker_reg = keras.wrappers.scikit_learn.KerasRegressor(build_model)
ker_reg.fit(X_train, y_train, epochs=100,
           validation_data=(X_val, y_val),
           callbacks=[keras.callbacks.EarlyStopping(patience=10)])
ker_reg.get_params()

Epoch 1/100


C:\Users\nick\AppData\Local\Temp\ipykernel_21864\2375457609.py:1: DeprecationWarning: KerasRegressor is deprecated, use Sci-Keras (https://github.com/adriangb/scikeras) instead. See https://www.adriangb.com/scikeras/stable/migration.html for help migrating.
  ker_reg = keras.wrappers.scikit_learn.KerasRegressor(build_model)


363/363 [==============================] - 1s 1ms/step - loss: 4.8865 - val_loss: 7.7963
Epoch 2/100
363/363 [==============================] - 0s 964us/step - loss: 4.6582 - val_loss: 7.1463
Epoch 3/100
363/363 [==============================] - 0s 946us/step - loss: 4.4252 - val_loss: 6.5346
Epoch 4/100
363/363 [==============================] - 0s 967us/step - loss: 4.1938 - val_loss: 5.9657
Epoch 5/100
363/363 [==============================] - 0s 919us/step - loss: 3.9678 - val_loss: 5.4428
Epoch 6/100
363/363 [==============================] - 0s 934us/step - loss: 3.7506 - val_loss: 4.9624
Epoch 7/100
363/363 [==============================] - 0s 967us/step - loss: 3.5434 - val_loss: 4.5246
Epoch 8/100
363/363 [==============================] - 0s 993us/step - loss: 3.3462 - val_loss: 4.1305
Epoch 9/100
363/363 [==============================] - 0s 957us/step - loss: 3.1586 - val_loss: 3.7721
Epoch 10/100
363/363 [==============================] - 0s 950us/step - loss: 2.9799 - 

363/363 [==============================] - 0s 943us/step - loss: 0.5584 - val_loss: 0.5128
Epoch 81/100
363/363 [==============================] - 0s 916us/step - loss: 0.5573 - val_loss: 0.5119
Epoch 82/100
363/363 [==============================] - 0s 898us/step - loss: 0.5563 - val_loss: 0.5109
Epoch 83/100
363/363 [==============================] - 0s 950us/step - loss: 0.5553 - val_loss: 0.5100
Epoch 84/100
363/363 [==============================] - 0s 953us/step - loss: 0.5543 - val_loss: 0.5091
Epoch 85/100
363/363 [==============================] - 0s 1ms/step - loss: 0.5533 - val_loss: 0.5082
Epoch 86/100
363/363 [==============================] - 0s 1ms/step - loss: 0.5523 - val_loss: 0.5073
Epoch 87/100
363/363 [==============================] - 0s 1ms/step - loss: 0.5514 - val_loss: 0.5064
Epoch 88/100
363/363 [==============================] - 0s 1ms/step - loss: 0.5504 - val_loss: 0.5056
Epoch 89/100
363/363 [==============================] - 0s 1ms/step - loss: 0.5495 - 

{'build_fn': <function __main__.build_model(n_hidden=1, n_neurons=30, learning_rate=0.003, input_shape=[8])>}

In [40]:
from scipy.stats import reciprocal
from sklearn.model_selection import RandomizedSearchCV
import numpy as np


In [41]:
param_distribs = {
    "n_hidden": [0, 1, 2, 3],
    "n_neurons": np.arange(1, 100)               .tolist(),
    "learning_rate": reciprocal(3e-4, 3e-2)      .rvs(1000).tolist(),
}

rnd_search_cv = RandomizedSearchCV(ker_reg, param_distribs, n_iter=10, cv=3, verbose=2)
rnd_search_cv.fit(X_train, y_train, epochs=100,
                  validation_data=(X_val, y_val),
                  callbacks=[keras.callbacks.EarlyStopping(patience=10)])

Fitting 3 folds for each of 10 candidates, totalling 30 fits
Epoch 1/100
242/242 [==============================] - 0s 1ms/step - loss: 2.3974 - val_loss: 0.6864
Epoch 2/100
242/242 [==============================] - 0s 1ms/step - loss: 0.5961 - val_loss: 0.4964
Epoch 3/100
242/242 [==============================] - 0s 992us/step - loss: 0.5359 - val_loss: 0.4751
Epoch 4/100
242/242 [==============================] - 0s 988us/step - loss: 0.5165 - val_loss: 0.4620
Epoch 5/100
242/242 [==============================] - 0s 1ms/step - loss: 0.5007 - val_loss: 0.4504
Epoch 6/100
242/242 [==============================] - 0s 1ms/step - loss: 0.4899 - val_loss: 0.4454
Epoch 7/100
242/242 [==============================] - 0s 1ms/step - loss: 0.4846 - val_loss: 0.4418
Epoch 8/100
242/242 [==============================] - 0s 1ms/step - loss: 0.4805 - val_loss: 0.4421
Epoch 9/100
242/242 [==============================] - 0s 990us/step - loss: 0.4780 - val_loss: 0.4391
Epoch 10/100
242/242 [==

242/242 [==============================] - 0s 1ms/step - loss: 0.4911 - val_loss: 0.4429
Epoch 10/100
242/242 [==============================] - 0s 1ms/step - loss: 0.4851 - val_loss: 0.4385
Epoch 11/100
242/242 [==============================] - 0s 1ms/step - loss: 0.4845 - val_loss: 0.4373
Epoch 12/100
242/242 [==============================] - 0s 1ms/step - loss: 0.4830 - val_loss: 0.4394
Epoch 13/100
242/242 [==============================] - 0s 1ms/step - loss: 0.4776 - val_loss: 0.4349
Epoch 14/100
242/242 [==============================] - 0s 1ms/step - loss: 0.4820 - val_loss: 0.4345
Epoch 15/100
242/242 [==============================] - 0s 1ms/step - loss: 0.4804 - val_loss: 0.4332
Epoch 16/100
242/242 [==============================] - 0s 1ms/step - loss: 0.4772 - val_loss: 0.4323
Epoch 17/100
242/242 [==============================] - 0s 1ms/step - loss: 0.4775 - val_loss: 0.4328
Epoch 18/100
242/242 [==============================] - 0s 1ms/step - loss: 0.4775 - val_loss: 

Epoch 58/100
242/242 [==============================] - 0s 1ms/step - loss: 0.5035 - val_loss: 0.5222
Epoch 59/100
242/242 [==============================] - 0s 1ms/step - loss: 0.5029 - val_loss: 0.5216
Epoch 60/100
242/242 [==============================] - 0s 1ms/step - loss: 0.5022 - val_loss: 0.5212
Epoch 61/100
242/242 [==============================] - 0s 1ms/step - loss: 0.5017 - val_loss: 0.5210
Epoch 62/100
242/242 [==============================] - 0s 1ms/step - loss: 0.5011 - val_loss: 0.5209
Epoch 63/100
242/242 [==============================] - 0s 1ms/step - loss: 0.5006 - val_loss: 0.5206
Epoch 64/100
242/242 [==============================] - 0s 1ms/step - loss: 0.5000 - val_loss: 0.5204
Epoch 65/100
242/242 [==============================] - 0s 1ms/step - loss: 0.4994 - val_loss: 0.5202
Epoch 66/100
242/242 [==============================] - 0s 1ms/step - loss: 0.4989 - val_loss: 0.5199
Epoch 67/100
242/242 [==============================] - 0s 1ms/step - loss: 0.4984

Epoch 59/100
242/242 [==============================] - 0s 1ms/step - loss: 0.4997 - val_loss: 0.9381
Epoch 60/100
242/242 [==============================] - 0s 1ms/step - loss: 0.4975 - val_loss: 0.9365
Epoch 61/100
242/242 [==============================] - 0s 1ms/step - loss: 0.4864 - val_loss: 0.9188
Epoch 62/100
242/242 [==============================] - 0s 1ms/step - loss: 0.4625 - val_loss: 0.9055
Epoch 63/100
242/242 [==============================] - 0s 1ms/step - loss: 0.4505 - val_loss: 0.9018
Epoch 64/100
242/242 [==============================] - 0s 1ms/step - loss: 0.4454 - val_loss: 0.9022
Epoch 65/100
242/242 [==============================] - 0s 1ms/step - loss: 0.4428 - val_loss: 0.9045
Epoch 66/100
242/242 [==============================] - 0s 1ms/step - loss: 0.4415 - val_loss: 0.9072
Epoch 67/100
242/242 [==============================] - 0s 1ms/step - loss: 0.4406 - val_loss: 0.9097
Epoch 68/100
242/242 [==============================] - 0s 1ms/step - loss: 0.4397

Epoch 65/100
242/242 [==============================] - 0s 1ms/step - loss: 0.5609 - val_loss: 0.6027
Epoch 66/100
121/121 [==============================] - 0s 612us/step - loss: 0.5510
[CV] END learning_rate=0.005432590230265343, n_hidden=2, n_neurons=94; total time=  17.6s
Epoch 1/100
242/242 [==============================] - 0s 1ms/step - loss: 5.0143 - val_loss: 6.2706
Epoch 2/100
242/242 [==============================] - 0s 1ms/step - loss: 5.0037 - val_loss: 6.2489
Epoch 3/100
242/242 [==============================] - 0s 1ms/step - loss: 4.9931 - val_loss: 6.2272
Epoch 4/100
242/242 [==============================] - 0s 1ms/step - loss: 4.9823 - val_loss: 6.2055
Epoch 5/100
242/242 [==============================] - 0s 1ms/step - loss: 4.9715 - val_loss: 6.1838
Epoch 6/100
242/242 [==============================] - 0s 1ms/step - loss: 4.9606 - val_loss: 6.1620
Epoch 7/100
242/242 [==============================] - 0s 1ms/step - loss: 4.9496 - val_loss: 6.1403
Epoch 8/100
242/

...

242/242 [==============================] - 0s 1ms/step - loss: 1.0230 - val_loss: 1.0174
Epoch 29/100
242/242 [==============================] - 0s 1ms/step - loss: 1.0211 - val_loss: 1.0202
Epoch 30/100
242/242 [==============================] - 0s 1ms/step - loss: 1.0200 - val_loss: 1.0173
Epoch 31/100
242/242 [==============================] - 0s 1ms/step - loss: 1.0186 - val_loss: 1.0184
Epoch 32/100
242/242 [==============================] - 0s 1ms/step - loss: 1.0171 - val_loss: 1.0190
Epoch 33/100
242/242 [==============================] - 0s 1ms/step - loss: 1.0158 - val_loss: 1.0178
Epoch 34/100
242/242 [==============================] - 0s 1ms/step - loss: 1.0142 - val_loss: 1.0222
Epoch 35/100
242/242 [==============================] - 0s 1ms/step - loss: 1.0134 - val_loss: 1.0187
Epoch 36/100
242/242 [==============================] - 0s 1ms/step - loss: 1.0119 - val_loss: 1.0173
Epoch 37/100
242/242 [==============================] - 0s 1ms/step - loss: 1.0110 - val_loss: 

Epoch 7/100
242/242 [==============================] - 0s 1ms/step - loss: 1.6736 - val_loss: 2.1156
Epoch 8/100
242/242 [==============================] - 0s 1ms/step - loss: 1.4360 - val_loss: 1.7796
Epoch 9/100
242/242 [==============================] - 0s 1ms/step - loss: 1.3053 - val_loss: 1.5589
Epoch 10/100
242/242 [==============================] - 0s 1ms/step - loss: 1.2243 - val_loss: 1.4108
Epoch 11/100
242/242 [==============================] - 0s 1ms/step - loss: 1.1668 - val_loss: 1.3007
Epoch 12/100
242/242 [==============================] - 0s 1ms/step - loss: 1.1208 - val_loss: 1.2201
Epoch 13/100
242/242 [==============================] - 0s 1ms/step - loss: 1.0790 - val_loss: 1.1513
Epoch 14/100
242/242 [==============================] - 0s 1ms/step - loss: 1.0321 - val_loss: 1.0733
Epoch 15/100
242/242 [==============================] - 0s 1ms/step - loss: 0.9798 - val_loss: 1.0202
Epoch 16/100
242/242 [==============================] - 0s 1ms/step - loss: 0.9507 - 

Epoch 18/100
242/242 [==============================] - 0s 1ms/step - loss: 0.8037 - val_loss: 2.1412
Epoch 19/100
242/242 [==============================] - 0s 1ms/step - loss: 0.7696 - val_loss: 2.0081
Epoch 20/100
242/242 [==============================] - 0s 1ms/step - loss: 0.7412 - val_loss: 1.9035
Epoch 21/100
242/242 [==============================] - 0s 1ms/step - loss: 0.7237 - val_loss: 1.8150
Epoch 22/100
242/242 [==============================] - 0s 1ms/step - loss: 0.7126 - val_loss: 1.7407
Epoch 23/100
242/242 [==============================] - 0s 1ms/step - loss: 0.7043 - val_loss: 1.6746
Epoch 24/100
242/242 [==============================] - 0s 1ms/step - loss: 0.6975 - val_loss: 1.6151
Epoch 25/100
242/242 [==============================] - 0s 1ms/step - loss: 0.6916 - val_loss: 1.5621
Epoch 26/100
242/242 [==============================] - 0s 1ms/step - loss: 0.6863 - val_loss: 1.5146
Epoch 27/100
242/242 [==============================] - 0s 1ms/step - loss: 0.6816

Epoch 36/100
242/242 [==============================] - 0s 1ms/step - loss: 0.5717 - val_loss: 0.5947
Epoch 37/100
242/242 [==============================] - 0s 1ms/step - loss: 0.5692 - val_loss: 0.5918
Epoch 38/100
242/242 [==============================] - 0s 1ms/step - loss: 0.5668 - val_loss: 0.5891
Epoch 39/100
242/242 [==============================] - 0s 1ms/step - loss: 0.5645 - val_loss: 0.5867
Epoch 40/100
242/242 [==============================] - 0s 1ms/step - loss: 0.5624 - val_loss: 0.5847
Epoch 41/100
242/242 [==============================] - 0s 1ms/step - loss: 0.5603 - val_loss: 0.5829
Epoch 42/100
242/242 [==============================] - 0s 1ms/step - loss: 0.5585 - val_loss: 0.5811
Epoch 43/100
242/242 [==============================] - 0s 1ms/step - loss: 0.5566 - val_loss: 0.5793
Epoch 44/100
242/242 [==============================] - 0s 1ms/step - loss: 0.5549 - val_loss: 0.5776
Epoch 45/100
242/242 [==============================] - 0s 1ms/step - loss: 0.5530

RandomizedSearchCV(cv=3,
                   estimator=<keras.wrappers.scikit_learn.KerasRegressor object at 0x00000293DE4ABE20>,
                   param_distributions={'learning_rate': [0.001683454924600351,
                                                          0.02390836445593178,
                                                          0.008731907739399206,
                                                          0.004725396149933917,
                                                          0.0006154014789262348,
                                                          0.0006153331256530192,
                                                          0.0003920021771415983,
                                                          0.01619845322936229,
                                                          0.004779156784872302,
                                                          0.007821074275112...
                                                          0.005021425736625637,
                                                          0.0005703073595961105,
                                                          0.001151888789941251,
                                                          0.001621231156394198,
                                                          0.0024505367684280487,
                                                          0.011155092541719619,
                                                          0.0007524347058135697,
                                                          0.0032032448128444043,
                                                          0.004591455636549438,
                                                          0.0003715541189658278, ...],
                                        'n_hidden': [0, 1, 2, 3],
                                        'n_neurons': [1, 2, 3, 4, 5, 6, 7, 8, 9,
                                                      10, 11, 12, 13, 14, 15,
                                                      16, 17, 18, 19, 20, 21,
                                                      22, 23, 24, 25, 26, 27,
                                                      28, 29, 30, ...]},
                   verbose=2)

In [42]:
rnd_search_cv.best_params_

{'n_neurons': 4, 'n_hidden': 1, 'learning_rate': 0.022174573948353458}

In [43]:
from sklearn.metrics import accuracy_score

In [48]:
model = rnd_search_cv.best_estimator_.model

In [56]:
model.evaluate(X_test, y_test)

162/162 [==============================] - 0s 624us/step - loss: 0.4648


0.46476876735687256